In [1]:
import io, sys, os, datetime, requests
import numpy as np
import pandas as pd
import xarray as xr
import shapely
import boto3
import geopandas as gpd

In [2]:
#!{sys.executable} -m pip install pip earthengine-api
#!{sys.executable} -m pip install pip geemap

In [3]:
import geemap
import ee
ee.Initialize()

In [4]:
# Data is CAMS EAC4, one calendar year, model level = 60, all hours,
# saved as netcdf and uploaded to this directory
camsdata = xr.open_dataset('cams-eac4_no2_sfc_2020.nc')

In [5]:
# define directory
out_dir = os.getcwd()
aws_s3_dir = "https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data"

In [6]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/geo_ref.csv')

# remove cities without tree cover data availability
#tml_not_available_cities = ['BRA-Salvador','MEX-Monterrey']
tml_not_available_cities = []
boundary_georef = boundary_georef[~boundary_georef['geo_name'].isin(tml_not_available_cities)].reset_index(drop=True)
boundary_georef

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Salvador,BRA-Salvador,ADM4union,ADM4,BRA-Salvador-ADM4,BRA,ADM4
1,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM2,COD,ADM2
2,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
3,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
4,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
5,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
6,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
7,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
8,Antananarivo,MDG-Antananarivo,ADM4union,ADM4,MDG-Antananarivo-ADM4,MDG,ADM4
9,Mexico_City,MEX-Mexico_City,ADM2union,ADM2,MEX-Mexico_City-ADM2,MEX,ADM2


In [7]:
def massfraction_to_concentration(massfraction):
    # input masses in kg, volumes in m^3
    # returns ug/m^3
    # 10^9 ug/kg
    # air density 1.223803 kg/m3 from https://confluence.ecmwf.int/display/UDOC/L60+model+level+definitions
    return massfraction * 1.223803 * 10**9

In [14]:
def exceedancedays(lon, lat):    
    localdata = camsdata.sel(latitude=lat, longitude=lon, method='nearest')
    conc = massfraction_to_concentration(localdata)
    dailymax = pd.DataFrame()
    dailymax['thedata'] = conc.to_array()[0]
    dailymax = dailymax.set_index(conc.time.to_index())
    dailymax = dailymax.resample('D').mean()
    return np.sum(dailymax.thedata >= 20.0)

In [15]:
cams_no2_indicator = pd.DataFrame() 

In [16]:
for i in range(0, len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
    
    print("\n geo_name: "+boundary_id_aoi)
    
    # process aoi level ------
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    shape = shapely.geometry.shape(boundary_geo['features'][0]['geometry'])
    centroid = shape.centroid
    clon, clat = centroid.coords[0]  # Breaks if multipolygon
    df = geemap.ee_to_pandas(boundary_geo_ee)
    
    df['exceedance_popfraction'] = exceedancedays(clon, clat)
    cams_no2_indicator = pd.concat([cams_no2_indicator, df])

0

 geo_name: BRA-Salvador-ADM4union

 boundary_id_aoi: BRA-Salvador-ADM4union
1

 geo_name: COD-Bukavu-ADM3union

 boundary_id_aoi: COD-Bukavu-ADM3union
2

 geo_name: COD-Uvira-ADM3union

 boundary_id_aoi: COD-Uvira-ADM3union
3

 geo_name: COG-Brazzaville-ADM4union

 boundary_id_aoi: COG-Brazzaville-ADM4union
4

 geo_name: COL-Barranquilla-ADM4union

 boundary_id_aoi: COL-Barranquilla-ADM4union
5

 geo_name: ETH-Addis_Ababa-ADM4union

 boundary_id_aoi: ETH-Addis_Ababa-ADM4union
6

 geo_name: ETH-Dire_Dawa-ADM3union

 boundary_id_aoi: ETH-Dire_Dawa-ADM3union
7

 geo_name: KEN-Nairobi-ADM3union

 boundary_id_aoi: KEN-Nairobi-ADM3union
8

 geo_name: MDG-Antananarivo-ADM4union

 boundary_id_aoi: MDG-Antananarivo-ADM4union
9

 geo_name: MEX-Mexico_City-ADM2union

 boundary_id_aoi: MEX-Mexico_City-ADM2union
10

 geo_name: MEX-Monterrey-ADM2union

 boundary_id_aoi: MEX-Monterrey-ADM2union


In [17]:
cams_no2_indicator

,geo_parent_name,geo_level,creation_date,geo_id,geo_name,exceedance_popfraction
0,BRA-Salvador,ADM4-union,2022-08-03,BRA-Salvador_ADM4-union_1,BRA-Salvador,0
0,COD-Bukavu,ADM3-union,2022-08-03,COD-Bukavu_ADM3-union_1,COD-Bukavu,0
0,COD-Uvira,ADM3-union,2022-08-03,COD-Uvira_ADM3-union_1,COD-Uvira,0
0,COG-Brazzaville,ADM4-union,2022-08-03,COG-Brazzaville_ADM4-union_1,COG-Brazzaville,2
0,COL-Barranquilla,ADM4-union,2022-08-03,COL-Barranquilla_ADM4-union_1,COL-Barranquilla,0
0,ETH-Addis_Ababa,ADM4-union,2022-08-03,ETH-Addis_Ababa_ADM4-union_1,ETH-Addis_Ababa,0
0,ETH-Dire_Dawa,ADM3-union,2022-08-03,ETH-Dire_Dawa_ADM3-union_1,ETH-Dire_Dawa,0
0,KEN-Nairobi,ADM3-union,2022-08-03,KEN-Nairobi_ADM3-union_1,KEN-Nairobi,0
0,MDG-Antananarivo,ADM4-union,2022-08-03,MDG-Antananarivo_ADM4-union_1,MDG-Antananarivo,0
0,MEX-Mexico_City,ADM2-union,2022-08-03,MEX-Mexico_City_ADM2-union_1,MEX-Mexico_City,0
